In [23]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px
pio.templates.default="plotly_dark"
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")
plt.style.use("fivethirtyeight")
from pandas_profiling import ProfileReport
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedKFold,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,StackingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from statsmodels.graphics.gofplots import qqplot
from category_encoders.target_encoder import TargetEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import RobustScaler,label_binarize
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import roc_curve
from scipy.stats import mode
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve,precision_recall_curve,confusion_matrix,classification_report
from pickle import dump

ModuleNotFoundError: No module named 'plotly'

In [2]:
data=pd.read_csv("C:/users/Shubham Mishra/OneDrive/Desktop/Bondora_raw.csv")
print(data.shape)
data.head()

(134529, 91)


C:\Users\Shubham Mishra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (34,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,...,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,Unnamed: 90
0,27-01-2020,F0660C80-83F3-4A97-8DA0-9C250112D6EC,659,11-06-2009 16:40,11-06-2009 16:40,0,0,115.0410,KARU,True,...,0.00,0.00,1,83.0852,0.0000,0.0,0,0.0,0.0,NaN
1,27-01-2020,978BB85B-1C69-4D51-8447-9C240104A3A2,654,10-06-2009 15:48,10-06-2009 15:48,0,0,140.6057,koort681,False,...,0.00,0.00,2,255.6467,258.6256,0.0,0,0.0,0.0,NaN
2,27-01-2020,EA44027E-7FA7-4BB2-846D-9C1F013C8A22,641,05-06-2009 19:12,05-06-2009 19:12,0,0,319.5580,0ie,True,...,116.35,414.07,0,0.0000,0.0000,0.0,0,0.0,0.0,NaN
3,27-01-2020,CE67AD25-2951-4BEE-96BD-9C2700C61EF4,668,13-06-2009 12:01,13-06-2009 12:01,0,0,57.5205,Alyona,True,...,0.00,0.00,1,134.2144,0.0000,0.0,0,0.0,0.0,NaN
4,27-01-2020,9408BF8C-B159-4D6A-9D61-9C2400A986E3,652,10-06-2009 10:17,10-06-2009 10:17,0,0,319.5582,Kai,True,...,0.00,0.00,1,146.9966,0.0000,0.0,0,0.0,0.0,NaN


In [3]:
drop_cols=['ReportAsOfEOD',
 'LoanId',
 'LoanNumber',
 'ListedOnUTC','DateOfBirth',
 'BiddingStartedOn','UserName', 'LanguageCode','LoanApplicationStartedDate','FirstPaymentDate','IncomeFromPrincipalEmployer', 'IncomeFromPension',
       'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare',
       'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther','LoanApplicationStartedDate','ApplicationSignedHour',
       'ApplicationSignedWeekday','ActiveScheduleFirstPaymentReached','ModelVersion','WorseLateCategory','PlannedPrincipalTillDate',"NextPaymentNr",'ProbabilityOfDefault',
          'ExpectedLoss', 'LossGivenDefault', 'ExpectedReturn']
print(len(drop_cols))
data.drop(drop_cols,axis=1,inplace=True)

29


KeyError: "['ModelVersion' 'NextPaymentNr' 'ProbabilityOfDefault' 'ExpectedLoss'\n 'LossGivenDefault' 'ExpectedReturn'] not found in axis"

In [4]:
nan=data.isnull().sum()[data.isnull().sum()>0]
print(nan/len(data))
nan=nan/len(data)
drop_cols=list(nan[nan>0.5].index)
print(len(drop_cols))

ContractEndDate                        0.561559
VerificationType                       0.000335
Gender                                 0.000335
MonthlyPayment                         0.049692
County                                 0.273844
City                                   0.072802
Education                              0.000335
MaritalStatus                          0.000335
NrOfDependants                         0.735373
EmploymentStatus                       0.001464
EmploymentDurationCurrentEmployer      0.006512
EmploymentPosition                     0.731731
WorkExperience                         0.728520
OccupationArea                         0.000639
HomeOwnershipType                      0.012280
DebtToIncome                           0.000335
FreeCash                               0.000335
PlannedPrincipalTillDate               0.595188
PlannedInterestTillDate                0.029302
LastPaymentOn                          0.070847
CurrentDebtDaysPrimary                 0

In [5]:
data.drop(drop_cols,axis=1,inplace=True)
print(data.shape)

(134529, 68)


In [6]:
date_cols=["LoanDate","MaturityDate_Original","MaturityDate_Last",'LastPaymentOn']
data.drop(date_cols,axis=1,inplace=True)

In [7]:
data.loc[data['Education']==1,"Education"]="Primary"
data.loc[data['Education']==2,"Education"]="Basic"
data.loc[data['Education']==3,"Education"]="Vocational"
data.loc[data['Education']==4,"Education"]="Secondary"
data.loc[data['Education']==5,"Education"]="Higher"
data.loc[data['Education'] == -1.0,"Education"] = 'Not_present'
data.loc[data['Education'] == 0.0,"Education"] = 'Not_present'

In [8]:
data.loc[data['EmploymentStatus']==1,"EmploymentStatus"]="Un-Employed"
data.loc[data['EmploymentStatus']==2,"EmploymentStatus"]="Partially-Employed"
data.loc[data['EmploymentStatus']==3,"EmploymentStatus"]="Fully-Employed"
data.loc[data['EmploymentStatus']==4,"EmploymentStatus"]="Self-Employed"
data.loc[data['EmploymentStatus']==5,"EmploymentStatus"]="Entrepreneur"
data.loc[data['EmploymentStatus']==6,"EmploymentStatus"]="Retiree"
data.loc[data['EmploymentStatus']==-1,"EmploymentStatus"]="Unknown"
data.loc[data['EmploymentStatus']==0,"EmploymentStatus"]="Unknown"

In [9]:
data.loc[data['Gender'] == 0.0,'Gender'] = 'Male'
data.loc[data['Gender'] == 1.0,'Gender']= 'Female'
data.loc[data['Gender'] == 2.0,'Gender'] = 'Unknown'

In [10]:
data.loc[data['HomeOwnershipType']==0,"HomeOwnershipType"]="Homeless"
data.loc[data['HomeOwnershipType']==1,"HomeOwnershipType"]="Owner"
data.loc[data['HomeOwnershipType']==2,"HomeOwnershipType"]='Living_with_parents'
data.loc[data['HomeOwnershipType']==3,"HomeOwnershipType"]='Tenant_pre_furnished_property'
data.loc[data['HomeOwnershipType']==4,"HomeOwnershipType"]="Tenant_unfurnished_property"
data.loc[data['HomeOwnershipType']==5,"HomeOwnershipType"]="Council_house"
data.loc[data['HomeOwnershipType']==6,"HomeOwnershipType"]="Joint_tenant"
data.loc[data['HomeOwnershipType']==7,"HomeOwnershipType"]="Joint_Ownership"
data.loc[data['HomeOwnershipType']==8,"HomeOwnershipType"]="Mortgage"
data.loc[data['HomeOwnershipType']==9,"HomeOwnershipType"]='Owner_with_encumbrance'
data.loc[data['HomeOwnershipType']==10,"HomeOwnershipType"]="Other"
data.loc[data['HomeOwnershipType']==-1,"HomeOwnershipType"]="Unknown"

In [11]:
data.loc[data['MaritalStatus']==-1,"MaritalStatus"]="Not_specified"
data.loc[data['MaritalStatus']==0,"MaritalStatus"]="Not_specified"
data.loc[data['MaritalStatus']==1,"MaritalStatus"]="Married"
data.loc[data['MaritalStatus']==2,"MaritalStatus"]="Cohabitant"
data.loc[data['MaritalStatus']==3,"MaritalStatus"]="Single"
data.loc[data['MaritalStatus']==4,"MaritalStatus"]="Divorced"
data.loc[data['MaritalStatus']==5,"MaritalStatus"]="Widow"

In [12]:
data.loc[data['OccupationArea'] == 1,'OccupationArea'] = 'Other'
data.loc[data['OccupationArea'] == 2,'OccupationArea'] = 'Mining'
data.loc[data['OccupationArea'] == 3,'OccupationArea'] = 'Processing'
data.loc[data['OccupationArea'] == 4,'OccupationArea'] = 'Energy'
data.loc[data['OccupationArea'] == 5,'OccupationArea'] = 'Utilities'
data.loc[data['OccupationArea'] == 6,'OccupationArea'] = 'Construction'
data.loc[data['OccupationArea'] == 7,'OccupationArea'] = 'Retail_and_wholesale'
data.loc[data['OccupationArea'] == 8,'OccupationArea'] = 'Transport_and_warehousing'
data.loc[data['OccupationArea'] == 9,'OccupationArea'] = 'Hospitality_and_catering'
data.loc[data['OccupationArea'] == 10,'OccupationArea'] = 'Info_and_telecom'
data.loc[data['OccupationArea'] == 11,'OccupationArea'] = 'Finance_and_insurance'
data.loc[data['OccupationArea'] == 12,'OccupationArea'] = 'Real_estate'
data.loc[data['OccupationArea'] == 13,'OccupationArea'] = 'Research'
data.loc[data['OccupationArea'] == 14,'OccupationArea'] = 'Administrative'
data.loc[data['OccupationArea'] == 15,'OccupationArea'] = 'Civil_service_and_military'
data.loc[data['OccupationArea'] == 16,'OccupationArea'] = 'Education'
data.loc[data['OccupationArea'] == 17,'OccupationArea'] = 'Healthcare_and_social_help'
data.loc[data['OccupationArea'] == 18,'OccupationArea'] = 'Art_and_entertainment'
data.loc[data['OccupationArea'] == 19,'OccupationArea'] = 'Agriculture_forestry_and_fishing'
data.loc[data['OccupationArea'] == -1.0,'OccupationArea'] = 'Not_specified'
data.loc[data['OccupationArea'] == 0.0,'OccupationArea'] = 'Not_specified'

In [13]:
data.loc[data['UseOfLoan'] == -1,'UseOfLoan'] = 'Not_set'
data.loc[data['UseOfLoan'] == 0,'UseOfLoan'] = 'Loan_consolidation'
data.loc[data['UseOfLoan'] == 1,'UseOfLoan'] = 'Real_estate'
data.loc[data['UseOfLoan'] == 2,'UseOfLoan'] = 'Home_improvement'
data.loc[data['UseOfLoan'] == 3,'UseOfLoan'] = 'Business'
data.loc[data['UseOfLoan'] == 4,'UseOfLoan'] = 'Education'
data.loc[data['UseOfLoan'] == 5,'UseOfLoan'] = 'Travel'
data.loc[data['UseOfLoan'] == 6,'UseOfLoan'] = 'Vehicle'
data.loc[data['UseOfLoan'] == 7,'UseOfLoan'] = 'Other'
data.loc[data['UseOfLoan'] == 8,'UseOfLoan'] = 'Health'
data.loc[data['UseOfLoan'] == 101,'UseOfLoan'] = 'Working_capital_financing'
data.loc[data['UseOfLoan'] == 102,'UseOfLoan'] = 'Purchase_of_machinery_equipment'
data.loc[data['UseOfLoan'] == 104,'UseOfLoan'] = 'Accounts_receivable_financing'
data.loc[data['UseOfLoan'] == 106,'UseOfLoan'] = 'Construction_finance'
data.loc[data['UseOfLoan'] == 107,'UseOfLoan'] = 'Acquisition_of_stocks'
data.loc[data['UseOfLoan'] == 108,'UseOfLoan'] = 'Acquisition_of_real_estate'
data.loc[data['UseOfLoan'] == 110,'UseOfLoan'] = 'Other_business'

In [14]:
data.loc[data['VerificationType'] == 0.0,'VerificationType'] = 'Not_set'
data.loc[data['VerificationType'] == 1.0,'VerificationType'] = 'Income_unverified'
data.loc[data['VerificationType'] == 2.0,'VerificationType'] = 'Income_unverified_crossref_phone'
data.loc[data['VerificationType'] == 3.0,'VerificationType'] = 'Income_verified'
data.loc[data['VerificationType'] == 4.0,'VerificationType'] = 'Income_expenses_verified'

In [15]:
data.loc[data['NewCreditCustomer'] == False,'NewCreditCustomer'] = 'Existing_credit_customer'
data.loc[data['NewCreditCustomer'] == True,'NewCreditCustomer'] = 'New_credit_Customer'

data.loc[data['Restructured'] == False,'Restructured']='No'
data.loc[data['Restructured'] == True,'Restructured']='Yes'

data.loc[data['RecoveryStage']==1,'RecoveryStage']='Collection'
data.loc[data['RecoveryStage']==0,'RecoveryStage']='Recovery'